In [1]:
!pip install keras
!pip install tensorflow
!pip install pydot
!pip install gensim

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
np.random.seed(13)

from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams

import gensim

Using TensorFlow backend.


In [3]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()

corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
V = len(tokenizer.word_index) + 1
V

3388

In [4]:
dim_embedddings = 128

# inputs
w_inputs = Input(shape=(1, ), dtype='int32')
w = Embedding(V, dim_embedddings)(w_inputs)

# context
c_inputs = Input(shape=(1, ), dtype='int32')
c  = Embedding(V, dim_embedddings)(c_inputs)
o = Dot(axes=2)([w, c])
o = Reshape((1,), input_shape=(1, 1))(o)
o = Activation('sigmoid')(o)

SkipGram = Model(inputs=[w_inputs, c_inputs], outputs=o)
SkipGram.summary()
SkipGram.compile(loss='binary_crossentropy', optimizer='adam')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       433664      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 128)       433664      input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [5]:
for _ in range(5):
    loss = 0.
    for i, doc in enumerate(tokenizer.texts_to_sequences(corpus)):
        data, labels = skipgrams(sequence=doc, vocabulary_size=V, window_size=5, negative_samples=5.)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if i<4:
            print(' ', i, len(x), y.shape)
        if x:
            loss += SkipGram.train_on_batch(x, y)

    print(loss)

  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
1078.3979634195566
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
743.0168305262923
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
687.0768266618252
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
659.7227480858564
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
635.0549664907157


In [6]:
f = open('vectors-sg-ns.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim_embedddings))
vectors = SkipGram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [7]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors-sg-ns.txt', binary=False)

In [8]:
w2v.most_similar(positive=['queen'])

[('hearts', 0.7493578791618347),
 ('king', 0.7314790487289429),
 ('turning', 0.651131272315979),
 ('wildly', 0.6425226926803589),
 ('tossing', 0.6305466294288635),
 ('cook', 0.6219874024391174),
 ('the', 0.6209454536437988),
 ('tarts', 0.6193763613700867),
 ('bread', 0.6036316156387329),
 ('trumpet', 0.6029201745986938)]

In [9]:
w2v.most_similar(positive=['alice'])

[('thought', 0.6477763056755066),
 ('cautiously', 0.6187073588371277),
 ('‘i’m', 0.6116552948951721),
 ('curious', 0.6014722585678101),
 ('rather', 0.5992931723594666),
 ('poor', 0.5799795389175415),
 ('feeling', 0.5739838480949402),
 ('nonsense', 0.5709330439567566),
 ('dreadfully', 0.5695661306381226),
 ('hastily', 0.5643173456192017)]

In [10]:
w2v.most_similar(positive=['the'])

[('hearts', 0.6456246972084045),
 ('queen', 0.6209454536437988),
 ('queen’s', 0.5883881449699402),
 ('mississippi', 0.5777144432067871),
 ('king', 0.5620546936988831),
 ('evidence', 0.5541610717773438),
 ('stole', 0.5521160960197449),
 ('cook', 0.551621675491333),
 ('received', 0.5495548844337463),
 ('verse', 0.5358551144599915)]